In [2]:
# импорт библиотек
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons

In [4]:
class MyKNeighbourClassifier:
    def __init__(self, n_neighbours: int = 5, metric: str = 'euclidean', weights: str = 'uniform'):
        
        # проверка на соответствие параметров
        if metric not in ['euclidean', 'manhattan']:
            raise ValueError(f'This type of metric is not supported: {metric}')
        if weights not in ['uniform', 'distance']:
            raise ValueError(f'This type of weights is not supported: {weights}')

        self.n_neigbours = n_neighbours
        self.metric = metric
        self.weights = weights

        self.X = None
        self.y = None

        def fit(X: np.ndarray, y: np.ndarray) -> None:
            self.X = X
            self.y = y

        def predict(self, X: np.ndarray) -> np.ndarray:
            result = []
            for point in X:
                # вычислим расстояние между тестовой точкой и всеми остальными
                if metric = 'euclidean':
                    distance = ((self.X - point) ** 2).sum(axis=1) ** 0.5
                elif metric = 'manhattan':
                    distance = abs(self.X - point).sum(axis=1)

                # для каждого высчитанного расстояния добавим метку класса конкретной точки
                points_with_dist = np.hstack((distance.reshape(-1, 1), self.y.reshape(-1, 1)))

                # получим k-ближайших точек в отсортированном порядке
                k_nearest = sorted(points_with_dist, key= lambda x: x[0])[:self.n_neighbours]

In [14]:
a = np.array([[1,2],[0, 0],[3, 4]])
b = np.array([2,2])
c = np.array([1,0, 2])
(a - b)

array([[-1,  0],
       [-2, -2],
       [ 1,  2]])

In [8]:
(a-b) ** 2

array([[1, 0],
       [4, 4],
       [1, 4]])

In [10]:
((a-b) ** 2).sum(axis=1) ** 0.5

array([1.        , 2.82842712, 2.23606798])

In [27]:
ans = (abs(a-b)).sum(axis=1).reshape(-1, 1)
ans

array([[1],
       [4],
       [3]])

In [28]:
d = np.hstack((ans, c.reshape(-1, 1)))
d

array([[1, 1],
       [4, 0],
       [3, 2]])

In [26]:
c

array([1, 0, 2])